## 🌠 [Day 4](https://adventofcode.com/2018/day/4)

In [0]:
import numpy as np
import collections

def string_to_state(x):
  if x == 'falls asleep':
    return -2
  elif x == 'wakes up':
    return -1
  else:
    return int(x[7:].split(' ', 1)[0])

def get_ordered_schedule(inputs):
  """Order the schedule by dates"""
  inputs = [(int(l[1:17].replace('-', '').replace(':', '').replace(' ', '')),  # time for sorting
             int(l[15:17]) - bool(int(l[12:14])) * 60,                         # minute
             string_to_state(l[19:])) for l in inputs]                         # state
  inputs = sorted(inputs, key=lambda x: x[:2])
  return inputs

def get_strategy(inputs):
  """Deduce sleeping times from the schedule and get both strategies"""
  schedule = get_ordered_schedule(inputs)
  guards_naps = collections.defaultdict(lambda: [])
  current_guard = None
  start_nap = None
  for day, minute, value in schedule:
    if value >= 0:
      # finish previous nap
      if start_nap is not None:
        guards_naps[current_guard].extend(list(range(start_nap, 60)))
      current_guard = value
      start_nap = None
    # awake
    elif value == -1:
      assert start_nap is not None
      guards_naps[current_guard].extend(list(range(start_nap, minute)))
      start_nap = None
    # asleep
    elif value == -2:
      start_nap = minute     
  # Strategy 1
  strat = np.array([(k, len(v), np.argmax(x), np.amax(x)) for k, v in guards_naps.items() 
                    for x in [np.bincount(v)]])
  worst_guard = np.argmax(strat[:, 1], axis=0)
  worst_guard = strat[worst_guard]
  strat1 = (worst_guard[0] * worst_guard[2], worst_guard)
  # Strategy 2
  worst_guard = np.argmax(strat[:, 3], axis=0)
  worst_guard = strat[worst_guard]
  strat2 = (worst_guard[0] * worst_guard[2], worst_guard)
  return strat1, strat2

In [2]:
with open("day4.txt", 'r') as f:
  inputs = f.read().splitlines()

strat1, strat2 = get_strategy(inputs)
print('Strategy 1: %d (details: %s)' % (strat1[0], strat1[1]))
print('Strategy 1: %d (details: %s)' % (strat2[0], strat2[1]))

Strategy 1: 63509 (details: [1549  567   41   16])
Strategy 1: 47910 (details: [1597  485   30   19])


In [3]:
#@title Visualize the Guard Shifts (Strategy 1 = Green, Strategy 2 = Cyan)
def visualize(inputs, guard1, minute1, guard2, minute2):
  schedule = get_ordered_schedule(inputs)
  schedule = [(str(day)[:-4], minute, value) for day, minute, value in schedule]
  days = np.unique([x[0] for x in schedule])
  naps = {k: collections.defaultdict(lambda: np.ones(59,)) for k in days}
  current_guard = None
  start_nap = None
  for day, minute, value in schedule:
    if value >= 0:
      # finish previous nap
      if start_nap is not None:
        naps[day][current_guard][start_nap:minute] = 0
      current_guard = value
      start_nap = None
    # awake
    elif value == -1:
      assert start_nap is not None
      naps[day][current_guard][start_nap:minute] = 0
      start_nap = None
    # asleep
    elif value == -2:
      start_nap = minute   
  # Display
  print('[   Date   ] Guard   #ID 01234%s59' % ('.' * 52))
  print('=' * 84)
  
  def fmt(in_):
    mn, t = in_
    s = '.' if t else '#'
    if mn == minute1: 
      return '\033[92m%s\033[0m' % s
    elif  mn == minute2: 
      return '\033[96m%s\033[0m' % s
    else:
      return s
    
  for k in sorted(naps.keys()):
    for guard, times in naps[k].items():
      print('[%s-%s-%s]' % (k[:4], k[4:6], k[6:8]),
            '%sGuard #%04d\033[0m' % ('\033[102m' if guard == guard1 else '\033[106m'
                                      if guard == guard2 else '\033[0m', guard), 
            ''.join(map(fmt, enumerate(times))))
  
visualize(inputs, 1549, 41, 1597, 30)

[   Date   ] Guard   #ID 01234....................................................59
[1518-03-02] Guard #0587 ....################################.....####..............
[1518-03-03] Guard #2113 ......................................................##...
[1518-03-04] Guard #2699 .............########################...#####..............
[1518-03-05] Guard #0593 ..............##################...........................
[1518-03-06] Guard #1543 ..............##...........................#######.........
[1518-03-07] Guard #2039 ....################################...#############.......
[1518-03-08] Guard #0587 .................................................##........
[1518-03-09] Guard #1601 ...........#######..............######################.....
[1518-03-10] Guard #1553 ...................###########################.............
[1518-03-11] Guard #1433 .................###################....###############....
[1518-03-12] Guard #2113 ..........############........##########